<h1>#####              Exercice  Aggréger des transactions         #####</h1>

In [21]:
import pymongo
import json

<h2>Connexion à la DataBase</h2>

In [22]:
try :
    client = pymongo.MongoClient('mongodb://localhost:27017/')

    db = client.db_agg
    db.transactions.drop()

    transactions = db.transactions
except :
    print("connexion error")

try :
    with open('json/transactions.json') as json_data:
        data = json.load(json_data)
except :
    print("impossible d'ouvrir les fichiers json")

try:
    db.transactions.insert_many(data)
except:
    print("Insert many error")



In [23]:
print(db.list_collection_names())

['transactions']


<h2>Q-1. Calculer le montant total des paiements ?</h2>

In [69]:
TotalPayment = { "$group" : {"_id" : None, "Total Payment" : { "$sum" : "$Payment.Total" }} }
for t in transactions.aggregate( [TotalPayment] ):
    print(t)

{'_id': None, 'Total Payment': 1230}


<h2>Q-2. Calculer le montant total par transactions (montant total du panier) ?</h2>

<h2>Q-3. Calculate total payments (Payment.Total) for each payment type (Payment.Type) ?</h2>

In [74]:
matching = {"$match":{"Payment":{"$ne": None}}}
TotalPayment = {"$group" : {"_id" : "$Payment.Type", "Total Payment" : { "$sum" : "$Payment.Total" }}}
for t in transactions.aggregate( [matching,TotalPayment] ):
    print(t)

{'_id': 'Debit-Card', 'Total Payment': 410}
{'_id': 'Credit-Card', 'Total Payment': 820}


<h2>Q-4. Trouver l'Id le plus élevé.</h2>

In [72]:
pipeline ={"$group": { "_id" : None , "Id": {"$max": "$Id"} }}

for t in transactions.aggregate([pipeline]):
    print(t)

{'_id': None, 'Id': 103}


<h2>Q-5. Find the max price (Transaction.price) ?</h2>

In [68]:
destroyArray = { "$unwind" : '$Panier' }
maxPrice = { "$group" : {"_id" : None , "max_price" : { "$max" : "$Panier.price" }} }
for t in transactions.aggregate( [destroyArray, maxPrice] ):
    print(t)

{'_id': None, 'max_price': 210}
